# Exploring Python 3's Asyncio by Example

http://www.giantflyingsaucer.com/blog/?p=5557

In [1]:
import asyncio
import imp

In [2]:
# imp.reload(asyncio)
@asyncio.coroutine
def my_coroutine1(seconds_to_sleep=3):
    print('my_coroutine sleeping for: {0} seconds'.format(seconds_to_sleep))
    yield from asyncio.sleep(seconds_to_sleep)

loop1 = asyncio.get_event_loop()
loop1.run_until_complete(
    asyncio.gather(my_coroutine1())
    )
# loop1.close()

my_coroutine sleeping for: 3 seconds


[None]

In [4]:
import asyncio
@asyncio.coroutine
def my_coroutine2(task_name, seconds_to_sleep=3):
    print('{} sleep {} seconds'.format(task_name, seconds_to_sleep))
    yield from asyncio.sleep(seconds_to_sleep)
    print('{} finished'.format(task_name))

loop2 = asyncio.get_event_loop()
tasks = [my_coroutine2('task1', 4),
         my_coroutine2('task2', 3),
         my_coroutine2('task3', 2)]
loop2.run_until_complete(asyncio.wait(tasks))
# loop.close()

task1 sleep 4 seconds
task2 sleep 3 seconds
task3 sleep 2 seconds
task3 finished
task2 finished
task1 finished


({<Task finished coro=<my_coroutine2() done, defined at <ipython-input-4-7c5d1aaa3b65>:2> result=None>,
  <Task finished coro=<my_coroutine2() done, defined at <ipython-input-4-7c5d1aaa3b65>:2> result=None>,
  <Task finished coro=<my_coroutine2() done, defined at <ipython-input-4-7c5d1aaa3b65>:2> result=None>},
 set())

In [5]:
@asyncio.coroutine
def my_coroutine3(future, task_name, seconds_to_sleep=3):
    print('{} sleep {} seconds'.format(task_name, seconds_to_sleep))
    yield from asyncio.sleep(seconds_to_sleep)
    future.set_result('{} finished'.format(task_name))

def got_result(future):
    print(future.result())

loop3 = asyncio.get_event_loop()
future1 = asyncio.Future()
future2 = asyncio.Future()

tasks = [
    my_coroutine3(future1, 'task1', 3),
    my_coroutine3(future2, 'task2', 1)
    ]
future1.add_done_callback(got_result)
future2.add_done_callback(got_result)

loop3.run_until_complete(asyncio.wait(tasks))
# loop3.close()

task1 sleep 3 seconds
task2 sleep 1 seconds
task2 finished
task1 finished


({<Task finished coro=<my_coroutine3() done, defined at <ipython-input-5-56a7bd8d567a>:1> result=None>,
  <Task finished coro=<my_coroutine3() done, defined at <ipython-input-5-56a7bd8d567a>:1> result=None>},
 set())

In [10]:
import aiohttp
import asyncio

@asyncio.coroutine
def fetch_page(url, pause=False):
    if pause:
        yield from asyncio.sleep(2)
    response = yield from aiohttp.request('GET', url)
    assert response.status == 200
    content = yield from response.read()
    print('URL: {}; Content: {}'.format(url, content[:20]))
    
loop4 = asyncio.get_event_loop()
tasks = [
    fetch_page('http://google.com'),
    fetch_page('http://cnn.com', True),
    fetch_page('http://twitter.com')
    ]
loop4.run_until_complete(asyncio.wait(tasks))
# loop4.close()

for task in tasks:
    print(task)

URL: http://google.com; Content: b'<!doctype html><html'
URL: http://twitter.com; Content: b'<!DOCTYPE html>\n<!--'
URL: http://cnn.com; Content: b'<!DOCTYPE html><html'
<generator object fetch_page at 0x105129510>
<generator object fetch_page at 0x1051297e0>
<generator object fetch_page at 0x105129558>


In [11]:
import asyncio

@asyncio.coroutine
def do_work(task_name, work_queue):
    while not work_queue.empty():
        queue_item = yield from work_queue.get()
        print('{} grabbed {}'.format(task_name, queue_item))
        yield from asyncio.sleep(0.5)
    
q = asyncio.Queue()
for x in range(20):
    q.put_nowait(x)
print(q)

loop5 = asyncio.get_event_loop()
tasks = [
    asyncio.async(do_work('task1', q)),
    asyncio.async(do_work('task2', q)),
    asyncio.async(do_work('task3', q))
    ]
loop5.run_until_complete(asyncio.wait(tasks))
# loop5.close()
    

<Queue maxsize=0 _queue=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]>
task1 grabbed 0
task2 grabbed 1
task3 grabbed 2
task1 grabbed 3
task2 grabbed 4
task3 grabbed 5
task1 grabbed 6
task2 grabbed 7
task3 grabbed 8
task1 grabbed 9
task2 grabbed 10
task3 grabbed 11
task1 grabbed 12
task2 grabbed 13
task3 grabbed 14
task1 grabbed 15
task2 grabbed 16
task3 grabbed 17
task1 grabbed 18
task2 grabbed 19


({<Task finished coro=<do_work() done, defined at <ipython-input-11-b904faa695a5>:3> result=None>,
  <Task finished coro=<do_work() done, defined at <ipython-input-11-b904faa695a5>:3> result=None>,
  <Task finished coro=<do_work() done, defined at <ipython-input-11-b904faa695a5>:3> result=None>},
 set())